# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
file_path_list = []
for root, dirs, files in os.walk(filepath):
    for f in files :
        file_path_list.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(file_path_list)
    print('{} files found in {}\n'.format(num_files, filepath))

    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [124]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    # Connect to local Apache Cassandra instance
    cluster = Cluster(['127.0.0.1']) 

    # Set session to connect andexecute queries.
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [125]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [126]:
# Set KEYSPACE
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query-1

In [127]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# CREATE TABLE:
# This CQL query creates song_in_session table which contains the following columns (with data type):
# * session_id INT, 
# * item_in_session INT, 
# * artist TEXT, 
# * song TEXT, 
# * length FLOAT
#
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# * session_id and item_in_session columns: are used as table's Primary Key (composite Partition Key). 

query = "CREATE TABLE IF NOT EXISTS song_in_session "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, \
                    PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [128]:
# INSERT data
# Set new file name.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_in_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign column elements in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [24]:
# SELECT statement:
# To answer Query-1, this CQL query 
# * matches session_id (=338) and item_in_session (=4) to 
# * return  artist, song and length from song_in_session table.

query = "SELECT artist, song, length \
                FROM song_in_session \
                WHERE   session_id = 338 AND \
                        item_in_session = 4"

try:
    songs = session.execute(query)
except Exception as e:
    print(e)

for row in songs:
    print (row.artist, row.song, row.length)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Query-2

In [130]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and 
# user (first and last name) for userid = 10, sessionid = 182

# CREATE TABLE
# This CQL query creates artist_in_session table which contains the following columns (with data type):
# * user_id INT,
# * session_id INT, 
# * artist TEXT, 
# * song TEXT, 
# * item_in_session INT, 
# * first_name TEXT,
# * last_name TEXT,
#
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# * user_id and session_id columns: are used as Composite Partition Key in table's Primary Key.
# * item_in_session column: is used as Clustering Key in table's Primary Key and allows sorting order of the data.

query = "CREATE TABLE IF NOT EXISTS artist_in_session "
query = query + "(  user_id int, \
                    session_id int, \
                    artist text, \
                    song text, \
                    item_in_session int, \
                    first_name text, \
                    last_name text, \
                    PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [131]:
# INSERT data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_in_session (user_id, \
                                                session_id, \
                                                artist, \
                                                song, \
                                                item_in_session, \
                                                first_name, \
                                                last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [25]:
# SELECT statement:
# To answer Query-2, this CQL query 
# * matches user_id (=10) and session_id (=182) to 
# * return  artist, song, first_name, and last_name (of user) from artist_in_session table.

query = "SELECT artist, song, first_name, last_name \
                    FROM artist_in_session \
                    WHERE   user_id = 10 AND \
                    session_id = 182"
try:
    artists = session.execute(query)
except Exception as e:
    print(e)

for row in artists:
    print (row.artist, row.song, row.first_name, row.last_name)

### Query-3

In [137]:
## Query 3: Give me every user name (first and last) in my music app history who listened 
# to the song 'All Hands Against His Own'

# CREATE TABLE
# CREATE TABLE
# This CQL query creates artist_in_session table which contains the following columns (with data type):
# * song TEXT,
# * user_id INT,
# * first_name TEXT,
# * last_name TEXT,
#
# To uniquely identify each row and allow efficient distribution in Cassandra cluster, 
# * song, user_id columns: are used as Composite Partition Key in table's Primary Key.

query = "CREATE TABLE IF NOT EXISTS user_and_song "
query = query + "(  song text, \
                    user_id int, \
                    first_name text, \
                    last_name text, \
                    PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [138]:
# INSERT data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_and_song (song, \
                                            user_id, \
                                            first_name, \
                                            last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [26]:
# SELECT statement:
# To answer Query-3, this CQL query 
# * matches song (=All Hands Against His Own) to 
# * return first_name and last_name (of users) from user_and_song table.

query = "SELECT first_name, last_name \
                    FROM user_and_song \
                    WHERE song = 'All Hands Against His Own'"
try:
    users = session.execute(query)
except Exception as e:
    print(e)

for row in users:
    print (row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [25]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE artist_in_session"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE user_and_song"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()